### the following is heavily influenced by this [blogpost](http://adventuresinmachinelearning.com/word2vec-tutorial-tensorflow/)

In [4]:
#import data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#data types
import collections

#import general libraries
import gc
import time
import random
import datetime

#deep learning 
import tensorflow as tf

/Users/waficel-assi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
#list_of_products = pd.read_csv('./data/subset_products_list_july30.csv')
df_product_embeddings = pd.read_csv('./data/subset_data_for_embeds_july30.csv')

In [8]:
df_product_embeddings.head()

,user_id,order_id,order_number,add_to_cart_order,product_id,product_name
0,1.0,473747.0,3.0,1.0,196,Soda
1,1.0,473747.0,3.0,2.0,12427,Original Beef Jerky
2,1.0,473747.0,3.0,3.0,10258,Pistachios
3,1.0,473747.0,3.0,4.0,25133,Organic String Cheese
4,1.0,473747.0,3.0,5.0,30450,Creamy Almond Butter


In [19]:
list_of_products = df_product_embeddings.product_name.tolist()

In [21]:
#number of unique products in dataset
print(len(set(list_of_products)))
n_products = len(set(list_of_products))

49688


In [22]:
#build skip-gram dataset
def embeddings_pre_processing(categories, n_categories):
    """Process raw inputs into a dataset."""
    
    count = [['UNK', -1]]
    count.extend(collections.Counter(categories).most_common(n_categories - 1))
    most_common_category_dict = dict()
    for category, _ in count:
        most_common_category_dict[category] = len(most_common_category_dict)
    
    idx_categories = list()
    unk_count = 0
    for category in categories:
        if category in most_common_category_dict:
            index = most_common_category_dict[category]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        idx_categories.append(index)
    
    count[0][1] = unk_count
    reversed_most_common_category_dict = dict(zip(most_common_category_dict.values(),
                                                   most_common_category_dict.keys()))
    
    return idx_categories, count, most_common_category_dict, reversed_most_common_category_dict

In [23]:
idx_products, count_products, most_common_products_dict, reversed_most_common_products_dict = embeddings_pre_processing(list_of_products, 20000)

In [24]:
data_index = 0
# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    
    return batch, context


In [25]:
batch_size = 300
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a context.
products_size = 20000

In [26]:
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [27]:
tf.reset_default_graph()
tf.get_default_graph()

In [28]:
graph = tf.Graph()
num_sampled = 20000

In [29]:
with graph.as_default():
        
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    embeddings = tf.get_variable('embeddings_graph', [products_size, embedding_size])
    embed_product_ids = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    # Construct the variables for the NCE loss
    nce_weights = tf.get_variable('nce_weights', [products_size, embedding_size])
    nce_biases = tf.get_variable('biases', [products_size], initializer=tf.zeros_initializer)

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed_product_ids,
                       num_sampled=num_sampled,
                       num_classes=products_size))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    # Add variable initializer.
    init = tf.global_variables_initializer()

In [ ]:
# #create embeddings variable and lookup tensor
# embeddings = tf.get_variable('embeddings_y', [20000, embedding_size])
# embed_product_ids = tf.nn.embedding_lookup(embeddings, train_inputs)

# #create weights and biases
# weights = tf.get_variable('weights_embeds', [20000, embedding_size])
# biases = tf.get_variable('biases', [20000], initializer=tf.zeros_initializer)
# hidden_out = tf.matmul(embed_product_ids, tf.transpose(weights)) + biases

# # convert train_context to a one-hot format
# train_one_hot = tf.one_hot(train_labels, 20000)
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hidden_out, labels=train_one_hot))
# # Construct the SGD optimizer using a learning rate of 0.1.
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cross_entropy)

# norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
# normalized_embeddings = embeddings / norm
# valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
# similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

# init = tf.global_variables_initializer()


In [30]:
def get_embeddings(graph, num_steps):
    with tf.Session(graph = graph) as session:
    
        # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        #tf.global_variables_initializer().run()

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(idx_products, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, nce_loss], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                  # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0

        final_embeddings = normalized_embeddings.eval()
        return final_embeddings

In [31]:
final_embeds_similarity_test = get_embeddings(graph, num_steps=1000)

Initialized
Average loss at step  0 :  13865.2353515625
